In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# SMILES 전처리
from rdkit import Chem

df = pd.read_csv("/content/drive/MyDrive/train.csv")
df['MOL'] = df['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))

In [ ]:
# seed
seed = 42

# define rmse
rmse = lambda x, y: np.mean((x - y) ** 2) ** 0.5

# features and targets
features = ["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea"]
mlm_target = "MLM"
hlm_target = "HLM"

# load data
df = pd.read_csv("./train.csv")
df["AlogP"] = np.where(pd.isna(df["AlogP"]), df["LogD"], df["AlogP"])

In [ ]:
# train
scores = []
reg_mlms = []
reg_hlms = []
kf = KFold(n_splits = 10, random_state = seed, shuffle = True)
for i, (train_index, valid_index) in enumerate(kf.split(df)):
    df_train = df.iloc[train_index]
    df_valid = df.iloc[valid_index]

    x_train = df_train[features].values
    y_mlm_train = df_train[mlm_target].values
    y_hlm_train = df_train[hlm_target].values

    x_valid = df_valid[features].values
    y_mlm_valid = df_valid[mlm_target].values
    y_hlm_valid = df_valid[hlm_target].values

    reg_mlm = RandomForestRegressor(random_state = seed)
    reg_mlm.fit(x_train, y_mlm_train)
    p_mlm = reg_mlm.predict(x_valid)

    reg_hlm = RandomForestRegressor(random_state = seed)
    reg_hlm.fit(x_train, y_hlm_train)
    p_hlm = reg_hlm.predict(x_valid)

    score = 0.5 * rmse(y_mlm_valid, p_mlm) + 0.5 * rmse(y_hlm_valid, p_hlm)

    reg_mlms.append(reg_mlm)
    reg_hlms.append(reg_hlm)
    scores.append(score)
    print(f"Fold {i+1:2d}: {score:.5f}")

print(f"CV score: {np.mean(scores):.5f}")

Fold  1: 34.16658
Fold  2: 34.03305
Fold  3: 33.21484
Fold  4: 34.63931
Fold  5: 32.44617
Fold  6: 33.18904
Fold  7: 31.83381
Fold  8: 31.91317
Fold  9: 31.68066
Fold 10: 33.10630
CV score: 33.02229
